In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import edward as ed
import numpy as np
import tensorflow as tf
import six
import os

In [3]:
from sklearn.linear_model import LogisticRegression

In [4]:
from edward.models import Normal,Empirical,Bernoulli,Categorical
from tensorflow.contrib import slim
from tensorflow.contrib.keras.api.keras.layers import Dense
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.distributions.python.ops.distribution_util import fill_lower_triangular

In [5]:
import matplotlib.pyplot as plt
import train_utils as util
import ddrp_gaussian as rp

In [6]:
%matplotlib inline

In [7]:
mtype = 'bpm'
iftype = 'HMC'

In [8]:
N = 10000  # number of data points
M = 2 # number of features
H = 3 # number of classes
B = 64 # batch size during training

In [9]:
s_mean = 0.
s_std = 1.

d_mean = 0.
d_std = 3.

noise_std = .01

In [10]:
Y,X = util.build_toy_dataset(mtype,N,M,H,s_std,s_mean,d_std)

In [11]:
lgr = LogisticRegression()
lgr.fit(X,Y)
print('Logistic regression accuracy: ',sum(lgr.predict(X)==Y)/N)

Logistic regression accuracy:  0.9719


In [12]:
X = X.astype(np.float32)
#Y = Y.astype(np.float32)

x_ph = tf.placeholder(tf.float32, [B,M])
y_ph = tf.placeholder(tf.int32,[B])
y_ph_ohe = tf.placeholder(tf.float32,[B,H]) 

In [13]:
# model
w = Normal(tf.zeros([H,M]),tf.ones([H,M]))
    
if iftype == 'HMC':
    qw = Empirical(params=tf.Variable(tf.random_normal([B,H,M])))
else:
    qw = Normal(tf.Variable(tf.random_normal([H,M])), tf.nn.softplus(tf.Variable(tf.random_normal([H,M]))))

y = Categorical(tf.nn.softmax(Normal(tf.matmul(x_ph,tf.transpose(w)), noise_std)))


In [14]:
# inference
scaling = float(N) / B
nprint = 1000
niter = 5000

if iftype == 'EP':
    inference = ed.KLpq({w:qw},data={y:y_ph})
elif iftype == 'VI':
    inference = ed.KLqp({w:qw},data={y:y_ph})
elif iftype == 'HMC':
    inference = ed.HMC({w:qw},data={y:y_ph})
else:
    print('invalid inference type')
    
inference.initialize(n_iter=niter,n_print=nprint,scale={y:scaling})


In [15]:
sess = ed.get_session()
tf.global_variables_initializer().run()

In [16]:
ii = 0
sess = ed.get_session()
for t in range(niter):
    x_batch,y_batch,ii = util.get_next_batch(X,B,ii,Y)
  
    info_dict = inference.update(feed_dict={x_ph:x_batch,y_ph:y_batch})
    inference.print_progress(info_dict)
    
    if t % nprint == 0:
        print('\n w mean:')
        print(sess.run(qw))

 1/64 [  1%]                                ETA: 34s | Acceptance Rate: nan
 w mean:
[[ 0.99775314 -0.42922267]
 [-3.21299434  0.64155132]
 [ 0.69233912 -0.11700325]]


train_utils.py:14: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if labels!=None:


InvalidArgumentError: indices = 64 is not in [0, 64)
	 [[Node: ScatterUpdate = ScatterUpdate[T=DT_FLOAT, Tindices=DT_INT32, _class=["loc:@Variable"], use_locking=true, _device="/job:localhost/replica:0/task:0/cpu:0"](Variable, iteration/read, cond/Merge)]]

Caused by op u'ScatterUpdate', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).sta

In [ ]:
ii = 0
acu = 0
y_test = tf.nn.softmax(tf.matmul(x_ph,tf.transpose(qw.loc)))
for i in range(int(np.ceil(N/B))):
    x_batch,y_batch,ii = util.get_next_batch(X,B,ii,Y)
    y_test_batch = sess.run(y_test,feed_dict={x_ph:x_batch,y_ph:y_batch})
    acu += sum(np.argmax(y_test_batch,axis=1)==y_batch)
print(acu*1./N)